In [272]:
import pandas as pd
import re 
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
import en_core_web_sm
import nltk
import os
from nltk.tokenize import word_tokenize
import emoji

In [273]:
from emosent import get_emoji_sentiment_rank

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [274]:
import sys
sys.path.append('../')
import functions
exec(open('./functions.py').read())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [472]:
df = pd.read_json('one_cleaned_df.json')
eng_df = df[df.language == 'en'].reset_index()
eng_df.head()

,level_0,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id
0,383,0,1323291682000000000,133,GreenerRetail,Who and what has made our greener retailing tr...,0,Planet Earth,05-Mar-2021,70,False,en,@zalando,NaN,NaN
1,384,1,1259761930000000000,1,fattybabycat,".@Zalando offers nothing more than scripted, c...",0,,05-Mar-2021,0,False,en,@zalando,NaN,NaN
2,385,2,835467350200000128,2296,piotrkarwatka,We've got a great talk with Fabian Wesner on h...,0,"Wrocław, Polska",05-Mar-2021,1347,False,en,@zalando,NaN,NaN
3,386,3,58485935,114,MissyDawn27586,@Zalando recieved wrong item... don’t know how...,0,,05-Mar-2021,27,False,en,@zalando,NaN,NaN
4,387,4,1059339104000000000,11725,CarolCooney7,@DrewLawDesign @Zalando have sent me a few de...,0,,05-Mar-2021,457,False,en,@zalando,NaN,NaN


In [473]:

#delete links, users
pat = r"(\\n)|(@\w*)|((https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}))"
eng_df.tweet = eng_df.tweet.str.replace(pat, '')
eng_df.tweet[3140]

C:\Anaconda\envs\uni\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


'Hey ! I ordered some products from your web shop. All items had “fast delivery” and are supposed to arrive today. Unfortunately you have not even shipped the parcel to me. This is not the first time that this happens.\n:-('

In [474]:

#replace emoticons with words
#SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

eng_df.tweet = eng_df.tweet.str.replace(r':-\)', ' smile')
eng_df.tweet = eng_df.tweet.str.replace(r':-\(', ' sad')
eng_df.tweet = eng_df.tweet.str.replace(r':-\/', ' confused')
eng_df.tweet = eng_df.tweet.str.replace(r':-P', ' playfullness')

eng_df.tweet[3140]

C:\Anaconda\envs\uni\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
C:\Anaconda\envs\uni\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
C:\Anaconda\envs\uni\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


'Hey ! I ordered some products from your web shop. All items had “fast delivery” and are supposed to arrive today. Unfortunately you have not even shipped the parcel to me. This is not the first time that this happens.\n sad'

In [475]:
eng_df.tweet[0]

'Who and what has made our greener retailing tracker this week? Step forward\xa0,\xa0, ,\xa0,\xa0, ,\xa0, ,\xa0,\xa0,\xa0, and\xa0\xa0 '

In [476]:
#delete \xa
eng_df.tweet = eng_df.tweet.str.replace('\xa0', '')
eng_df.tweet[0]

'Who and what has made our greener retailing tracker this week? Step forward,, ,,, ,, ,,,, and '

In [477]:
eng_df.tweet = eng_df.tweet.str.replace('&amp', '')
eng_df.tweet = eng_df.tweet.str.replace('\n', '')
eng_df.tweet[3140]


'Hey ! I ordered some products from your web shop. All items had “fast delivery” and are supposed to arrive today. Unfortunately you have not even shipped the parcel to me. This is not the first time that this happens. sad'

In [478]:
#to lower case
eng_df.tweet = eng_df.tweet.str.lower()
eng_df.tweet[3140]

'hey ! i ordered some products from your web shop. all items had “fast delivery” and are supposed to arrive today. unfortunately you have not even shipped the parcel to me. this is not the first time that this happens. sad'

In [479]:
#covert hashtags to the normal text
eng_df.tweet = eng_df.tweet.str.replace(r'#([^\s]+)', r'\1')
eng_df.tweet[3140]

C:\Anaconda\envs\uni\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


'hey ! i ordered some products from your web shop. all items had “fast delivery” and are supposed to arrive today. unfortunately you have not even shipped the parcel to me. this is not the first time that this happens. sad'

In [263]:
import demoji
demoji.download_codes()

... OK (Got response in 0.47 seconds)
Writing emoji data to C:\Users\home\.demoji\codes.json ...
... OK


In [480]:
#delete numbers
eng_df.tweet = [strip_numeric(c) for c in eng_df.tweet]
eng_df.tweet[7]

'  zalando thinks im big and/or pregnant and suggests me lovely maternity clothes ❤️❤️❤️❤️❤️❤️❤️ thanks i love it ❤️❤️❤️❤️'

In [481]:
#replacing emojies with descriptions '❤️-> red heart'
eng_df.tweet = [demoji.replace_with_desc(c, ' ') for c in eng_df.tweet]
eng_df.tweet[9]

"new  redezign for circularity pilot product is out now! all products of the pilot are equipped with our  a digital tag which saves all product data. once scanned, you are able to discover the product's history through a digital product site  down arrow  recycling symbol  "

In [482]:
eng_df.tweet[7]

'  zalando thinks im big and/or pregnant and suggests me lovely maternity clothes  red heart  red heart  red heart  red heart  red heart  red heart  red heart  thanks i love it  red heart  red heart  red heart  red heart '

In [483]:
#delete punctuation
eng_df.tweet = [strip_punctuation(c) for c in eng_df.tweet]
eng_df.tweet[7]

'  zalando thinks im big and or pregnant and suggests me lovely maternity clothes  red heart  red heart  red heart  red heart  red heart  red heart  red heart  thanks i love it  red heart  red heart  red heart  red heart '

In [484]:
eng_df.tweet = [remove_stopwords(c) for c in eng_df.tweet]
eng_df.tweet[7]

'zalando thinks im big pregnant suggests lovely maternity clothes red heart red heart red heart red heart red heart red heart red heart thanks love red heart red heart red heart red heart'

In [485]:
eng_df.tweet = [strip_short(c) for c in eng_df.tweet]
eng_df.tweet[7]

'zalando thinks big pregnant suggests lovely maternity clothes red heart red heart red heart red heart red heart red heart red heart thanks love red heart red heart red heart red heart'

In [486]:
eng_df.tweet = [strip_multiple_whitespaces(c) for c in eng_df.tweet]
eng_df.tweet[7]

'zalando thinks big pregnant suggests lovely maternity clothes red heart red heart red heart red heart red heart red heart red heart thanks love red heart red heart red heart red heart'

In [489]:
nlp = spacy.load("en_core_web_sm")

In [503]:
eng_df['lem'] = [nlp(c) for c in eng_df.tweet]

In [496]:
eng_df.lem[1]

offers scripted cordial replies instead actual customer service waste time resources shipping protocols create huge inconveniences hurt environment won’t long til dominates fail

In [504]:
eng_df['lemma'] = [" ".join([token.lemma_ for token in t]) for t in eng_df.lem]
eng_df.lemma[7]

'zalando think big pregnant suggest lovely maternity clothe red heart red heart red heart red heart red heart red heart red heart thank love red heart red heart red heart red heart'

In [510]:
eng_df[['tweet', 'lemma']]

,tweet,lemma
0,greener retailing tracker week step forward,greener retailing tracker week step forward
1,offers scripted cordial replies instead actual...,offer script cordial reply instead actual cust...
2,got great talk fabian wesner new venture roq t...,get great talk fabian wesner new venture roq t...
3,recieved wrong item don’t know item cheaper it...,recieve wrong item do n’t know item cheap item...
4,sent deliveries paper bags,send delivery paper bag
...,...,...
109982,recent aresmonitor updates zalando rewriteoffs...,recent aresmonitor update zalando rewriteoffsp...
109983,i’ve able hit zalando sleepy face extension ch...,I ’ve able hit zalando sleepy face extension c...
109984,gear enjoy favorite sport high quality sportsw...,gear enjoy favorite sport high quality sportsw...
109985,restocking zalando accounts today wanna miss n...,restock zalando account today wanna miss night...
